# Value Investing Automation

## 1. Load Libraries

In [1]:
import csv
import requests
import numpy as np
from time import sleep

## 2. Set some initial parameters

In [2]:
# Minimum market capitalization, to weed out small-cap stocks
min_cap = 300000000
# API key required to access teh financial modeling prep services
api_key = 'api key here'

## 3. Grab a list of NYSE & NASDAQstock tickers

Grabbed from a generic .csv file containing almost all stocks listed on the NYSE

In [3]:
stocks_raw = []

Grab NYSE listed stocks

In [4]:
ticker_to_name = {}

with open("nyse-listed.csv", mode='r') as csv_file:
    # open the .csv and iterate through the rows
    csv_reader = csv.DictReader(csv_file)
    line = 0
    for row in csv_reader:
        line += 1
        if '$' in row['ACT Symbol']:
            # '$' character is found in some weird listings in the .csv file.
            # Just ignore these listings
            continue
        if line == 0:
            # first (zeroeth) line is just column labels, ignore them
            continue
        
        # put the ticker into a list if it matches the above criteria.
        # Called an 'ACT Symbol' for some reason in this .csv
        stocks_raw.append(row['ACT Symbol'])
        ticker_to_name[row['ACT Symbol']] = row['Company Name']
        

Grab NASDAQ listed stocks that aren't found on the NYSE

In [5]:
with open('nasdaqlisted.txt', mode='r') as txt_file:
    for line in txt_file:
        ticker = line.split("|")[0]
        company_name = line.split("|")[1].split(" - ")[0]
        if ticker == "Symbol":
            continue    # First line is just column labels, ignore it
        if ticker in ticker_to_name:
            continue    # If it's listed also on the NYSE skip it to avoid duplicates
        
        ticker_to_name[ticker] = company_name
        stocks_raw.append(ticker)

Print the tickers, if requested

In [6]:
#for ticker in stocks_raw:
#    print(ticker)

## 4. Define functions to grab the market capitalization (CAP), return on invested capital (ROIC), and price to earnings ratio (PE)

In [7]:
key_metrics_dict = {}

In [8]:
def getCAP(stock, api_key):
    if not stock in key_metrics_dict:
        try:
            # Try to grab the information directly
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?limit=1&apikey={api_key}").json()
        except:
            # If attempt fails, it's most likely we've exceeded the number of API calls allowed
            # within a specific amount of time. Wait a few minutes and then try again.
            print("getCAP: too many requests! Pausing for 5 minutes")
            sleep(300)
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?limit=1&apikey={api_key}").json()
        key_metrics_dict[stock] = key_metrics
    else:
        key_metrics = key_metrics_dict[stock]
        
    try:
        # try to return the CAP. It is sometimes possible that the CAP is not available
        # for a given ticker. If that is the case, return Nothing
        return key_metrics[0]['marketCap']
    except:
        return None

# ============================= Same logic applies on lower 2 functions ======================================== #
    
def getROIC(stock, api_key):
    if not stock in key_metrics_dict:
        try:
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?limit=1&apikey={api_key}").json()
        except:
            print("getROIC: too many requests! Pausing for 5 minutes")
            sleep(300)
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?limit=1&apikey={api_key}").json()
        key_metrics_dict[stock] = key_metrics
    else:
        key_metrics = key_metrics_dict[stock]
        
    try:
        return key_metrics[0]['roic']
    except:
        return None
    

def getPE(stock, api_key):
    if not stock in key_metrics_dict:
        try:
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?period=annual&limit=1&apikey={api_key}").json()
        except:
            print("getPE: too many requests! Pausing for 5 minutes")
            sleep(300)
            key_metrics = requests.get(f"https://financialmodelingprep.com/api/v3/key-metrics/{stock}?period=annual&limit=1&apikey={api_key}").json()
        key_metrics_dict[stock] = key_metrics
    else:
        key_metrics = key_metrics_dict[stock]
    
    try:
        return key_metrics[0]['peRatio']
    except:
        return None

# Perform some tests to ensure that the above functions work,
# grab some basic information on Apple Inc.
print("(Example) Apple market cap:\t" + str(getCAP('AAPL', api_key)))
print("(Example) Apple ROIC:\t\t" + str(getROIC('AAPL', api_key)))
print("(Example) Apple PE:\t\t" + str(getPE('AAPL', api_key)))

(Example) Apple market cap:	1996361343006.3572
(Example) Apple ROIC:		0.30705825278265964
(Example) Apple PE:		34.773150493918536


## 5. Remove the tickers with market caps below the pre-specified threshold

In [9]:
stocks = []
for s in stocks_raw:
    # Grab the market cap for a ticker
    cap = getCAP(s, api_key)
    
    # If the market capitalization cannot be found, don't add it. Just skip it.
    if not cap:
        continue
        
    # If the ticker's market cap is above the threshold, add it.
    if cap >= min_cap:
        stocks.append(s)

getCAP: too many requests! Pausing for 5 minutes


## 6. Grab the PE & ROIC for remaining tickers

In [10]:
stockdat = []
for s in stocks:
    sdat = {}
    roic = getROIC(s, api_key)    # Grab the ROIC
    pe = getPE(s, api_key)        # Grab the PE ratio
    
    # If either the ROIC or PE ratio cannot be found for a
    # ticker, ignore it.
    if (roic is None) or (pe is None):
        continue
        
    # Add the pe, roic, and name to a dictionary object and
    # append it to a list of stock data
    sdat['pe'] = pe
    sdat['roic'] = roic
    sdat['name'] = s
    
    stockdat.append(sdat)

## 7. Sort the tickers by general rank

Set some initial parameters to further sort results

In [11]:
filter_results = True

pe_min = 0.0             # minimum price to earnings ratio
pe_max = 60.0            # maximum price to earnings ratio

roic_min = 0.06          # minimum ROIC
roic_max = 9999999999    # maximum ROIC

In [15]:
# Generate an argsort list to organize the PE ratios from lowest to highest
pe_list_argsort = np.argsort(np.array([x['pe'] for x in stockdat]))

# Generate an argsort list to organize the ROIC from highest to lowest
roic_list_argsort = np.flip(np.argsort(np.array([x['roic'] for x in stockdat])))

# Attach the PE ratio & ROIC rankings to each ticker.
# Also calculate a "general" rank by adding each PE & ROIC
# ranking.
for i in range(len(stockdat)):
    stockdat[pe_list_argsort[i]]['pe_rank'] = i
    stockdat[roic_list_argsort[i]]['roic_rank'] = i
    
for i in range(len(stockdat)):
    stockdat[i]['rank'] = stockdat[i]['pe_rank'] + stockdat[i]['roic_rank']
    stockdat[i]['Full Name'] = ticker_to_name[stockdat[i]['name']]
    stockdat[i]['Website'] =  f"https://finance.yahoo.com/quote/{stockdat[i]['name']}?p={stockdat[i]['name']}&.tsrc=fin-srch"

# Generate an argsort list to organize stock tickers by "general rank"
rank_argsort = np.argsort(np.array([x['rank'] for x in stockdat]))

#for i in range(10):
    #roic_sorted = sorted(stockdat, key = lambda x: x['roic_rank'])
    #pe_sorted = sorted(stockdat, key = lambda x: x['pe_rank'])
    #print(roic_sorted[i]['roic'], roic_sorted[i]['roic_rank'])
    #print(pe_sorted[i]['pe'], pe_sorted[i]['pe_rank'])
    #print(stockdat[roic_list_argsort[i]]['roic'], stockdat[roic_list_argsort[i]]['roic_rank'])
    
    
# Print all stock tickers in order of "general" rank, assuming
# they're within the thresholds
print("Name \t|\t rank \t|\t PE \t|\t ROIC \t|\t Website")
print("=====================================================================================================")
for i in range(len(stockdat)):
    sdat = stockdat[rank_argsort[i]]
    
    if(filter_results):
        if (sdat['pe'] <= pe_min) or (sdat['pe'] >= pe_max):
            continue
        if (sdat['roic'] <= roic_min) or (sdat['roic'] >= roic_max):
            continue
    
    
    print(str(sdat['name']) +
          "\t|\t" +
          str(sdat['rank']) +
          "\t|\t" +
          "{0:.3f}".format(sdat['pe']) +
          "x\t|\t" +
          "{0:.2%}".format(sdat['roic']) +
          "\t|\t" +
          ticker_to_name[sdat['name']] + " " +
          f"https://finance.yahoo.com/quote/{sdat['name']}?p={sdat['name']}&.tsrc=fin-srch")

Name 	|	 rank 	|	 PE 	|	 ROIC 	|	 Website
XBIT	|	999	|	1.318x	|	94.95%	|	XBiotech Inc. https://finance.yahoo.com/quote/XBIT?p=XBIT&.tsrc=fin-srch
UZA	|	1015	|	1.591x	|	56.30%	|	United States Cellular Corporation 6.95% Senior Notes due 2060 https://finance.yahoo.com/quote/UZA?p=UZA&.tsrc=fin-srch
NVO	|	1059	|	3.710x	|	72.87%	|	Novo Nordisk A/S Common Stock https://finance.yahoo.com/quote/NVO?p=NVO&.tsrc=fin-srch
QFIN	|	1064	|	0.521x	|	27.76%	|	360 DigiTech, Inc. https://finance.yahoo.com/quote/QFIN?p=QFIN&.tsrc=fin-srch
OMAB	|	1065	|	0.937x	|	28.67%	|	Grupo Aeroportuario del Centro Norte S.A.B. de C.V. https://finance.yahoo.com/quote/OMAB?p=OMAB&.tsrc=fin-srch
LXRX	|	1071	|	2.612x	|	35.22%	|	Lexicon Pharmaceuticals, Inc. https://finance.yahoo.com/quote/LXRX?p=LXRX&.tsrc=fin-srch
HHR	|	1075	|	0.739x	|	26.66%	|	HeadHunter Group PLC https://finance.yahoo.com/quote/HHR?p=HHR&.tsrc=fin-srch
TSM	|	1081	|	1.217x	|	27.28%	|	Taiwan Semiconductor Manufacturing Company Ltd. https://finance.yahoo.c

## Generate & export a .csv file for further analysis

In [16]:
with open('value_stock_analysis.csv', mode='w', newline='') as csv_file:
    fieldnames = ['name', 'pe', 'roic', 'pe_rank', 'roic_rank', 'rank', 'Full Name', 'Website']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for row in stockdat:
        writer.writerow(row)